<a href="https://colab.research.google.com/github/BaoQuoc285/sentiment-classification/blob/main/UIT_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install datasets transformers==4.28.0
!pip install transformers datasets
!pip install transformers[torch]
!pip install accelerate -U
!pip install evaluate



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.2 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import transformers
import evaluate
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import f1_score
from datasets import load_dataset
from datasets import Dataset

from gensim.utils import simple_preprocess
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix

import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import  DataLoader
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler, TrainingArguments, Trainer

import re

from transformers import get_linear_schedule_with_warmup, AutoTokenizer, AutoModel, logging

import warnings
warnings.filterwarnings("ignore")

logging.set_verbosity_error()

In [ ]:
def seed_everything(seed_value):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed_everything(86)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 15
N_SPLITS = 4

In [ ]:
# pip install sacremoses

In [ ]:
# model_name = "vinai/phobert-large"
# model_name = "nam7197/vi-nli-xlm-roberta-base"
#model_name = "vinai/phobert-base-v2"
#model_name = 'Fsoft-AIC/videberta-base'
# model_name1 = 'xlm-roberta-base'
model_name = 'symanto/xlm-roberta-base-snli-mnli-anli-xnli'

# tokenizer1 = AutoTokenizer.from_pretrained("xlm-roberta-base")
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base' )

(…)-xnli/resolve/main/tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

(…)li-anli-xnli/resolve/main/tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

(…)nli/resolve/main/special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
def get_data(path):
    df = pd.read_json(path)
    df = df.transpose()

    df.drop(columns=['context','domain'], inplace=True)
    df = df.reset_index(drop=True)
    return df


data = get_data('/content/ise-dsc01-warmup.json')

In [ ]:
data

,claim,verdict,evidence
0,"Trong năm nay, hai địa phương dẫn đầu và gần n...",SUPPORTED,Năm nay hai địa phương có lượng hồ sơ dẫn đầu ...
1,FAA có hơn 10.000 nhân viên,NEI,"Cư dân của Port Isabel, một thị trấn cách địa ..."
2,"Do đó, chị đã quyết định mở hàng ốc nhồi, hơn ...",SUPPORTED,"Vì vậy, sau khi trở lại Sài Gòn, chị đã quyết ..."
3,Số người mắc chứng tâm lý PTSD tăng mạnh sau m...,NEI,Kinh nghiệm hỗ trợ người khác đã giúp Ivanov n...
4,Bộ Tài chính sẽ chịu trách nhiệm hướng dẫn các...,SUPPORTED,Các địa phương phải sử dụng hiệu quả dưới sự h...
...,...,...,...
995,Điều khiến du khách ấn tượng khi ghé Bãi Bùi c...,REFUTED,Điều khiến du khách ấn tượng khi ghé Bãi Bùi c...
996,Chặng Launceston-Sydney có vấn đề về kĩ thuật ...,SUPPORTED,"""Do vấn đề thời tiết, kỹ thuật nên chặng Launc..."
997,Gueorguiev được phép trò chuyện với người khác...,SUPPORTED,Gueorguiev được phép trò chuyện với mọi người ...
998,Tỉnh Gia Lai đã và đang huy động người dân cân...,NEI,"Năm 2022-2023, toàn tỉnh Gia Lai có trên 30.00..."


In [ ]:

import plotly.express as px

In [ ]:
labels, frequencies = np.unique(data.verdict.values, return_counts = True)

fig = px.pie(values=frequencies,
             names=labels,
             title='Languages distribution',
             color_discrete_sequence=px.colors.sequential.Plotly3)

fig.show()

In [ ]:
import pandas as pd

# Assuming 'data' is a DataFrame with a 'verdict' column

label_mapping = {'SUPPORTED': 0, 'REFUTED': 1, 'NEI': 2}

def label_to_correct(label):
    if label is not None:
        return label_mapping.get(label.strip(), -1)
    else:
        return label_mapping.get(label, -1)
data['verdict'] = data['verdict'].apply(label_to_correct)


In [ ]:
data

,claim,verdict,evidence
0,"Trong năm nay, hai địa phương dẫn đầu và gần n...",0,Năm nay hai địa phương có lượng hồ sơ dẫn đầu ...
1,FAA có hơn 10.000 nhân viên,2,"Cư dân của Port Isabel, một thị trấn cách địa ..."
2,"Do đó, chị đã quyết định mở hàng ốc nhồi, hơn ...",0,"Vì vậy, sau khi trở lại Sài Gòn, chị đã quyết ..."
3,Số người mắc chứng tâm lý PTSD tăng mạnh sau m...,2,Kinh nghiệm hỗ trợ người khác đã giúp Ivanov n...
4,Bộ Tài chính sẽ chịu trách nhiệm hướng dẫn các...,0,Các địa phương phải sử dụng hiệu quả dưới sự h...
...,...,...,...
995,Điều khiến du khách ấn tượng khi ghé Bãi Bùi c...,1,Điều khiến du khách ấn tượng khi ghé Bãi Bùi c...
996,Chặng Launceston-Sydney có vấn đề về kĩ thuật ...,0,"""Do vấn đề thời tiết, kỹ thuật nên chặng Launc..."
997,Gueorguiev được phép trò chuyện với người khác...,0,Gueorguiev được phép trò chuyện với mọi người ...
998,Tỉnh Gia Lai đã và đang huy động người dân cân...,2,"Năm 2022-2023, toàn tỉnh Gia Lai có trên 30.00..."


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_labels = np.unique(data['verdict'])
class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=data['verdict'])
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

In [ ]:
class_weights

tensor([0.7153, 5.0505, 0.7123])

In [ ]:
import string

punctuations=list(string.punctuation)

In [ ]:
# punctuations

In [ ]:
def normalizer(text):

  def remove_meaningless_token(s):
    return re.sub(r'\b(jeez|yeah|uhhuh|uh|oh|huh|ừ|à)\b',' ',s)
  def uncased(s):
    return s.lower()

  def fix_whitespaces(s):
    return re.sub(r'\s+',' ',s)

  def remove_punctuations(s):
    chs=[]
    for ch in s:
      if ch not in punctuations:
        chs.append(ch)
      else:
        chs.append(" ")
    return "".join(chs)

  return fix_whitespaces(remove_meaningless_token(uncased(remove_punctuations(text)))).strip()

In [ ]:
data['claim']=data['claim'].apply(normalizer)
data['evidence']=data['evidence'].apply(normalizer)

In [ ]:
data

,claim,verdict,evidence
0,trong năm nay hai địa phương dẫn đầu và gần nh...,0,năm nay hai địa phương có lượng hồ sơ dẫn đầu ...
1,faa có hơn 10 000 nhân viên,2,cư dân của port isabel một thị trấn cách địa đ...
2,do đó chị đã quyết định mở hàng ốc nhồi hơn mộ...,0,vì vậy sau khi trở lại sài gòn chị đã quyết đị...
3,số người mắc chứng tâm lý ptsd tăng mạnh sau m...,2,kinh nghiệm hỗ trợ người khác đã giúp ivanov n...
4,bộ tài chính sẽ chịu trách nhiệm hướng dẫn các...,0,các địa phương phải sử dụng hiệu quả dưới sự h...
...,...,...,...
995,điều khiến du khách ấn tượng khi ghé bãi bùi c...,1,điều khiến du khách ấn tượng khi ghé bãi bùi c...
996,chặng launceston sydney có vấn đề về kĩ thuật ...,0,do vấn đề thời tiết kỹ thuật nên chặng launces...
997,gueorguiev được phép trò chuyện với người khác...,0,gueorguiev được phép trò chuyện với mọi người ...
998,tỉnh gia lai đã và đang huy động người dân cân...,2,năm 2022 2023 toàn tỉnh gia lai có trên 30 000...


In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
# smote = SMOTE(sampling_strategy='auto')  # Có thể điều chỉnh sampling_strategy nếu cần
# X_resampled, y_resampled = smote.fit_resample(X, y)

In [ ]:
k = 5  # Số fold (tùy chọn)
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
data['kfold'] = np.nan
for fold, (_, val_) in enumerate(skf.split(X=data[['claim','evidence']], y=data['verdict'])):
    data.loc[val_, "kfold"] = fold


In [ ]:
data

,claim,verdict,evidence,kfold
0,trong năm nay hai địa phương dẫn đầu và gần nh...,0,năm nay hai địa phương có lượng hồ sơ dẫn đầu ...,0.0
1,faa có hơn 10 000 nhân viên,2,cư dân của port isabel một thị trấn cách địa đ...,4.0
2,do đó chị đã quyết định mở hàng ốc nhồi hơn mộ...,0,vì vậy sau khi trở lại sài gòn chị đã quyết đị...,0.0
3,số người mắc chứng tâm lý ptsd tăng mạnh sau m...,2,kinh nghiệm hỗ trợ người khác đã giúp ivanov n...,2.0
4,bộ tài chính sẽ chịu trách nhiệm hướng dẫn các...,0,các địa phương phải sử dụng hiệu quả dưới sự h...,0.0
...,...,...,...,...
995,điều khiến du khách ấn tượng khi ghé bãi bùi c...,1,điều khiến du khách ấn tượng khi ghé bãi bùi c...,4.0
996,chặng launceston sydney có vấn đề về kĩ thuật ...,0,do vấn đề thời tiết kỹ thuật nên chặng launces...,4.0
997,gueorguiev được phép trò chuyện với người khác...,0,gueorguiev được phép trò chuyện với mọi người ...,1.0
998,tỉnh gia lai đã và đang huy động người dân cân...,2,năm 2022 2023 toàn tỉnh gia lai có trên 30 000...,1.0


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df,tokenizer, max_len=128):
        self.df = df
        self.max_len = max_len
        self.label_encoder = LabelEncoder()
        self.tokenizer = tokenizer
        self.df['verdict'] = self.label_encoder.fit_transform(self.df['verdict'])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        evidence = self.df.iloc[idx]['evidence']
        claim = self.df.iloc[idx]['claim']
        label = self.df.iloc[idx]['verdict']

        # Encode evidence and claim using PhoBERT tokenizer
        encoding = self.tokenizer.encode_plus(
            evidence,
            claim,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_token_type_ids=False,
            return_tensors='pt',
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_masks': attention_mask,
            'targets': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
def prepare_loaders(df, fold):
    df_train = df[df["kfold"] != fold].reset_index(drop=True)
    df_eval = df[df["kfold"] == fold].reset_index(drop=True)

    train_dataset = CustomDataset(df_train, tokenizer, max_len=120)
    valid_dataset = CustomDataset(df_eval, tokenizer, max_len=120)

    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
    eval_loader = DataLoader(valid_dataset, batch_size=4, shuffle=True, num_workers=2)

    return train_loader, eval_loader

In [ ]:
def train(model, criterion, optimizer, train_loader):
    model.train()
    losses = []
    correct = 0

    for data in train_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_masks'].to(device)
        targets = data['targets'].to(device)

        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        loss = criterion(outputs, targets)
        _, pred = torch.max(outputs, dim=1)

        correct += torch.sum(pred == targets)
        losses.append(loss.item())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        lr_scheduler.step()

    print(f'Train Accuracy: {correct.double()/len(train_loader.dataset)} Loss: {np.mean(losses)}')

In [ ]:

from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return accuracy, precision, recall, f1

In [ ]:
def eval1( valid_loader):

    model.eval()
    losses = []
    correct = 0
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        data_loader = valid_loader
        for data in data_loader:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_masks'].to(device)
            targets = data['targets'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, pred = torch.max(outputs, dim=1)

            loss = criterion(outputs, targets)
            correct += torch.sum(pred == targets)
            losses.append(loss.item())

            all_predictions.extend(pred.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
        accuracy, precision, recall, f1 = calculate_metrics(all_targets, all_predictions)
        accuracy = correct.double() / len(valid_loader.dataset)
        f1 = f1_score(all_targets, all_predictions, average='weighted')  # Calculate F1 score

        print(f'Test Accuracy: {accuracy} Loss: {np.mean(losses)} F1 Score: {f1} Precision: {precision} Recall: {recall}')
        return accuracy, f1, np.mean(losses)

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel

class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.4)
        self.fc1 = nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size // 2)
        self.fc2 = nn.Linear(self.bert.config.hidden_size // 2, n_classes)
        self.leakyrelu = nn.LeakyReLU(0.1)  # Add Leaky ReLU activation
        self.softmax = torch.nn.Softmax(dim=1)
        nn.init.normal_(self.fc1.weight, std=0.1)
        nn.init.normal_(self.fc1.bias, 0.1)
        nn.init.normal_(self.fc2.weight, std=0.1)
        nn.init.normal_(self.fc2.bias, 0.1)

    def forward(self, input_ids, attention_mask):
        last_hidden_state, output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False
        )

        x = self.drop(output)
        x = self.fc1(x)
        x = self.leakyrelu(x)  # Apply Leaky ReLU activation
        x = self.fc2(x)
        x = self.softmax(x)
        return x


In [ ]:
model = SentimentClassifier(n_classes=3).to(device)

In [ ]:
model

SentimentClassifier(
  (bert): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
       

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/uit_challenge')

In [ ]:
os.getcwd()

'/content/drive/MyDrive/uit_challenge'

In [ ]:
from torch.nn.modules import loss
for fold in range(skf.n_splits):
    print(f'-----------Fold: {fold+1} ------------------')
    train_loader, valid_loader = prepare_loaders(data, fold=fold)
    criterion = nn.CrossEntropyLoss(weight = class_weights)

    # Recommendation by BERT: lr: 5e-5, 2e-5, 3e-5
    # Batchsize: 16, 32

    optimizer = AdamW(model.parameters(), lr=1e-5,weight_decay=0.02)

    lr_scheduler = get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps=0,
                num_training_steps=len(train_loader)*EPOCHS
            )
    best_f1 = 0
    loss1 = 5
    for epoch in range(EPOCHS):
        print(f'Epoch {epoch+1}/{EPOCHS}')
        print('-'*30)

        train(model, criterion, optimizer, train_loader)
        val_acc,valf1,lss = eval1(valid_loader)
        if valf1 > best_f1 and lss < loss1 :
            torch.save(model.state_dict(), f'phobert_fold{fold+1}.pth')
            best_f1 = valf1
            loss1 = lss



-----------Fold: 1 ------------------
Epoch 1/15
------------------------------


In [ ]:
# model.to('cpu')
# def predict(claim, context):
#   # Tokenize claim và context
#   model.load_state_dict(torch.load('/content/drive/MyDrive/check point/phobert_fold2.pth',map_location=torch.device('cpu')))
#   inputs = tokenizer(
#       claim,
#       context,
#       padding=True,
#       truncation=True,
#       return_tensors="pt"
#   )
#   input_ids = inputs["input_ids"].to('cpu')
#   attention_mask = inputs["attention_mask"].to('cpu')


#     # Dự đoán
#   with torch.no_grad():

#       outputs = model(input_ids, attention_mask=attention_mask)
#       probabilities = torch.softmax(outputs, dim=1)
#       print(probabilities)

#   # Lấy các lớp và xác suất dự đoán tương ứng
#   labels = ['neutral', 'support']
#   predicted_label = labels[torch.argmax(probabilities, axis=1).item()]
#   predicted_probabilities = probabilities.tolist()[0]

#   return predicted_label, predicted_probabilities


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# model.load_state_dict(torch.load('/content/phobert_fold4.pth',map_location=torch.device('cpu')))

In [ ]:
# def get_data(path):
#     df = pd.read_json(path)
#     df = df.transpose()
#     df.columns = ['verdict','claim', 'evidence']
#     df = df.reset_index(drop=True)
#     return df


# data = get_data('/content/label_nei_full.json')

In [ ]:
# print(data['claim'][i])
# print(data['evidence'][i])

In [ ]:
# claim = data['claim'][i]
# context = data['evidence'][i]

In [ ]:
# label,predit_prob = predict(claim,context)

In [ ]:
# label

In [ ]:
# predit_prob